Install dependencies and load Data files

In [ ]:
# 
# !pip install -U -q PyDrive

# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# # Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)
# print('success!')

# import os
# import zipfile

# # Download helper functions file
# helper_file = drive.CreateFile({'id': '16HW-z9Y1tM3gZ_vFpJAuwUDohz91Aac-'})
# helper_file.GetContentFile('helpers.py')
# print('helper file downloaded! (helpers.py)')

# # Download training data file of Meme Propaganda Classification
# data_file = drive.CreateFile({'id': '1JIyYSCE8qVfWoekHijm2lHE8WsHNwUQE'})
# data_file.GetContentFile('memeClassification_train.csv')
# print('Training data for Memes Propaganda Multi-Label Classification downloaded! (memeClassification_train.csv)')

# # Download testing data file of Meme Propaganda Classification
# data_file = drive.CreateFile({'id': '13LdgdCHXz2Z7GawaR5mgDro2eG_YUo5r'})
# data_file.GetContentFile('memeClassification_test.csv')
# print('Training data for Memes Propaganda Multi-Label Classification downloaded! (memeClassification_test.csv)')



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install tensorflow-addons==0.5.0
import os
import collections

# OR easier way: Change the Working Directory to the drive folder itself
%cd "/content/drive/MyDrive/685-NLP_Project"
print(os.getcwd())

/content/drive/MyDrive/685-NLP_Project
/content/drive/MyDrive/685-NLP_Project


Tokenize and Load Pretrained BERT

Reference: https://www.kaggle.com/code/javaidnabi/toxic-comment-classification-using-bert/notebook

In [ ]:
import pandas as pd
from transformers import BertForSequenceClassification, AdamW, BertConfig
from torch.utils.data import TensorDataset, random_split
from transformers import BertTokenizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import sys
import numpy as np
import time
import datetime
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
#import tensorflow_addons as tfa
from helpers import tokenize_and_format, flat_accuracy

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

DATA_COLUMN = 'text'
LABEL_COLUMNS = ["Loaded Language",
              "Name calling/Labeling",
              "Smears",
              "Appeal to fear/prejudice",
              "Black-and-white Fallacy/Dictatorship",
              "Exaggeration/Minimisation",
              "Slogans",
              "Misrepresentation of Someone's Position (Straw Man)",
              "Obfuscation, Intentional vagueness, Confusion",
              "Presenting Irrelevant Data (Red Herring)",
              "Whataboutism",
              "Causal Oversimplification",
              "Appeal to authority",
              "Flag-waving",
              "Doubt",
              "Thought-terminating cliché",
              "Bandwagon",
              "Reductio ad hitlerum",
              "Repetition",
              "Glittering generalities (Virtue)"]

trainingData = pd.read_csv("Propaganda_Meme_Dataset/propagandaMemeClassification_training.csv")
testingData = pd.read_csv("Propaganda_Meme_Dataset/propagandaMemeClassification_testing.csv")
trainingData.loc[1]

text                                                   This is not an accident!
Loaded Language                                                             0.0
Name calling/Labeling                                                       0.0
Smears                                                                      0.0
Appeal to fear/prejudice                                                    0.0
Black-and-white Fallacy/Dictatorship                                        0.0
Exaggeration/Minimisation                                                   0.0
Slogans                                                                     0.0
Misrepresentation of Someone's Position (Straw Man)                         0.0
Obfuscation, Intentional vagueness, Confusion                               0.0
Presenting Irrelevant Data (Red Herring)                                    0.0
Whataboutism                                                                0.0
Causal Oversimplification               

In [ ]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, labels=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            labels: (Optional) [string]. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.labels = labels

class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self, input_ids, input_mask, segment_ids, label_ids, is_real_example=True):
      self.input_ids = input_ids
      self.input_mask = input_mask
      self.segment_ids = segment_ids
      self.label_ids = label_ids,
      self.is_real_example=is_real_example

def create_examples(df, labels_available=True):
    """Creates examples for the training and dev sets."""
    examples = []
    ctr = 0
    for (i, row) in enumerate(df.values):
      guid = ctr
      ctr+=1
      text = row[0]
      labels = row[1:]
      examples.append(
          InputExample(guid=guid, text_a=text, labels=labels))
    return examples


train_examples = create_examples(trainingData)
#Nuprint(train_examples[1])
test_examples = create_examples(testingData)


In [ ]:
def convert_examples_to_features(examples,  max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    features = []
    for (ex_index, example) in enumerate(examples):
        print(example.text_a)
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        
        labels_ids = []
        for label in example.labels:
            labels_ids.append(int(label))

        if ex_index < 0:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %s)" % (example.labels, labels_ids))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_ids=labels_ids))
    return features


In [ ]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
    """Creates a classification model."""
    model = modeling.BertModel(
        config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    # In the demo, we are doing a simple classification task on the entire
    # segment.
    #
    # If you want to use the token-level output, use model.get_sequence_output()
    # instead.
    output_layer = model.get_pooled_output()

    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable(
        "output_weights", [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
        "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):
        if is_training:
            # I.e., 0.1 dropout
            output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        
        # probabilities = tf.nn.softmax(logits, axis=-1) ### multiclass case
        probabilities = tf.nn.sigmoid(logits)#### multi-label case
        
        tf.logging.info("labels_check1:{}".format(labels))
        labels = tf.cast(labels, tf.float32)
        tf.logging.info("labels_check2:{}".format(labels))
        tf.logging.info("num_labels:{};logits:{};labels:{}".format(num_labels, logits, labels))
        per_example_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)
        loss = tf.reduce_mean(per_example_loss)

        # probabilities = tf.nn.softmax(logits, axis=-1)
        # log_probs = tf.nn.log_softmax(logits, axis=-1)
        #
        # one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        #
        # per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        # loss = tf.reduce_mean(per_example_loss)

        return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(bert_config, num_labels, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        #tf.logging.info("*** Features ***")
        #for name in sorted(features.keys()):
        #    tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]
        
        is_real_example = None
        if "is_real_example" in features:
             is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)
        else:
             is_real_example = tf.ones(tf.shape(label_ids), dtype=tf.float32)

        is_training = (mode == tf.estimator.ModeKeys.TRAIN)

        (total_loss, per_example_loss, logits, probabilities) = create_model(
            bert_config, is_training, input_ids, input_mask, segment_ids, label_ids,
            num_labels, use_one_hot_embeddings)

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        scaffold_fn = None
        if init_checkpoint:
            (assignment_map, initialized_variable_names
             ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
            if use_tpu:

                def tpu_scaffold():
                    tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
                    return tf.train.Scaffold()

                scaffold_fn = tpu_scaffold
            else:
                tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        tf.logging.info("**** Trainable Variables ****")
        for var in tvars:
            init_string = ""
            if var.name in initialized_variable_names:
                init_string = ", *INIT_FROM_CKPT*"
            #tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,init_string)

        output_spec = None
        if mode == tf.estimator.ModeKeys.TRAIN:

            train_op = optimization.create_optimizer(
                total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op,
                scaffold=scaffold_fn)
        elif mode == tf.estimator.ModeKeys.EVAL:

            def metric_fn(per_example_loss, label_ids, probabilities, is_real_example):

                # change metric to f1 score of each class and take avg
                # metric = tf.metrics.F1Score(num_classes=3, threshold=0.05, average="weighted")
                # metric.update_state(label_ids, probabilities)
                # eval_dict = {}
                # eval_dict['f1_score'] = metric.result()
                # return eval_dict

                logits_split = tf.split(probabilities, num_labels, axis=-1)
                label_ids_split = tf.split(label_ids, num_labels, axis=-1)
                # metrics change to auc of every class
                eval_dict = {}
                for j, logits in enumerate(logits_split):
                    label_id_ = tf.cast(label_ids_split[j], dtype=tf.int32)
                    current_auc, update_op_auc = tf.metrics.auc(label_id_, logits)
                    eval_dict[str(j)] = (current_auc, update_op_auc)
                eval_dict['eval_loss'] = tf.metrics.mean(values=per_example_loss)
                return eval_dict

                ## original eval metrics
                # predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                # accuracy = tf.metrics.accuracy(
                #     labels=label_ids, predictions=predictions, weights=is_real_example)
                # loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
                # return {
                #     "eval_accuracy": accuracy,
                #     "eval_loss": loss,
                # }

            eval_metrics = metric_fn(per_example_loss, label_ids, probabilities, is_real_example)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metric_ops=eval_metrics,
                predictions={"probabilities": probabilities},
                scaffold=scaffold_fn)
        else:
            print("mode:", mode, "true_labels:", label_ids, "probabilities:", probabilities)
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={"probabilities": probabilities},
                scaffold=scaffold_fn)
        return output_spec

    return model_fn

In [ ]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 20.0
NUM_CLASSES = 20

# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

In [ ]:
OUTPUT_DIR = os.getcwd()+'baseline/output'
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    keep_checkpoint_max=1,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [ ]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_ids)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples, len(LABEL_COLUMNS)], dtype=tf.int32), 
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
    When running eval/predict on the TPU, we need to pad the number of examples
    to be a multiple of the batch size, because the TPU requires a fixed batch
    size. The alternative is to drop the last batch, which is bad because it means
    the entire output data won't be generated.
    We use this class instead of `None` because treating `None` as padding
    battches could cause silent errors.
    """


In [ ]:
def convert_single_example(ex_index, example, max_seq_length,
                           tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    if isinstance(example, PaddingInputExample):
        return InputFeatures(
            input_ids=[0] * max_seq_length,
            input_mask=[0] * max_seq_length,
            segment_ids=[0] * max_seq_length,
            label_ids=0,
            is_real_example=False)

    tokens_a = tokenizer.tokenize(example.text_a)
    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    labels_ids = []
    for label in example.labels:
        labels_ids.append(int(label))


    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_ids=labels_ids,
        is_real_example=True)
    return feature


def file_based_convert_examples_to_features(
        examples, max_seq_length, tokenizer, output_file):
    """Convert a set of `InputExample`s to a TFRecord file."""

    writer = tf.io.TFRecordWriter(output_file)

    for (ex_index, example) in enumerate(examples):
        #if ex_index % 10000 == 0:
            #tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

        feature = convert_single_example(ex_index, example,
                                         max_seq_length, tokenizer)

        def create_int_feature(values):
            f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
            return f

        features = collections.OrderedDict()
        features["input_ids"] = create_int_feature(feature.input_ids)
        features["input_mask"] = create_int_feature(feature.input_mask)
        features["segment_ids"] = create_int_feature(feature.segment_ids)
        features["is_real_example"] = create_int_feature(
            [int(feature.is_real_example)])
        if isinstance(feature.label_ids, list):
            label_ids = feature.label_ids
        else:
            label_ids = feature.label_ids[0]
        features["label_ids"] = create_int_feature(label_ids)

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()


def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    name_to_features = {
        "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
        "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
        "label_ids": tf.FixedLenFeature([NUM_CLASSES], tf.int64),
        "is_real_example": tf.FixedLenFeature([], tf.int64),
    }

    #print(labels_ids)

    def _decode_record(record, name_to_features):
        """Decodes a record to a TensorFlow example."""
        example = tf.parse_single_example(record, name_to_features)

        # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
        # So cast all int64 to int32.
        for name in list(example.keys()):
            t = example[name]
            if t.dtype == tf.int64:
                t = tf.to_int32(t)
            example[name] = t

        return example

    def input_fn(params):
        """The actual input function."""
        batch_size = params["batch_size"]

        # For training, we want a lot of parallel reading and shuffling.
        # For eval, we want no shuffling and parallel reading doesn't matter.
        d = tf.data.TFRecordDataset(input_file)
        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: _decode_record(record, name_to_features),
                batch_size=batch_size,
                drop_remainder=drop_remainder))

        return d

    return input_fn


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [ ]:
#from pathlib import Path

#Create the Training DF
print(os.getcwd())
train_file = os.getcwd()+'/baseline/train.tf_record'
print(train_file)
print("Training DF already exists: " + str(os.path.exists(train_file)))
# Create file if doesn't exist
if not os.path.exists(train_file):
    created_file = open(train_file, 'w+')
    created_file.close()

#Create Test DF
test_file = os.getcwd()+'/baseline/test.tf_record'
print(test_file)
print("Testing DF already exists: " + str(os.path.exists(test_file)))
# Create file if doesn't exist
if not os.path.exists(test_file):
    created_file = open(test_file, 'w+')
    created_file.close()


/content/drive/MyDrive/685-NLP_Project
/content/drive/MyDrive/685-NLP_Project/baseline/train.tf_record
Training DF already exists: True
/content/drive/MyDrive/685-NLP_Project/baseline/test.tf_record
Testing DF already exists: True


In [ ]:
pip install tensorflow==1.15.0

In [ ]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_examples) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Transform Training Data into appropriate format for BERT
file_based_convert_examples_to_features(
            train_examples, MAX_SEQ_LENGTH, tokenizer, train_file)
tf.logging.info("***** Training Data Transformed *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", BATCH_SIZE)
tf.logging.info("  Num steps = %d", num_train_steps)

# train_features = convert_examples_to_features( train_examples, MAX_SEQ_LENGTH, tokenizer)
# train_input_fn = input_fn_builder( features=train_features, seq_length=MAX_SEQ_LENGTH, is_training=True, drop_remainder=False)
train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)


# Transform Test Data into appropriate format for BERT
file_based_convert_examples_to_features(
            test_examples, MAX_SEQ_LENGTH, tokenizer, test_file)
tf.logging.info("***** Test Data Transformed *****")
tf.logging.info("  Num examples = %d", len(test_examples))

test_input_fn = file_based_input_fn_builder(
    input_file=test_file,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=True)


INFO:tensorflow:***** Training Data Transformed *****
INFO:tensorflow:  Num examples = 688
INFO:tensorflow:  Batch size = 32
INFO:tensorflow:  Num steps = 430
INFO:tensorflow:***** Test Data Transformed *****
INFO:tensorflow:  Num examples = 200


In [ ]:
!pip install bert-tensorflow==1.0.1

     |████████████████████████████████| 67 kB 5.4 MB/s 


In [ ]:
import os
import collections
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling

In [ ]:
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
BERT_VOCAB= '../input/uncased-l12-h768-a12/vocab.txt'
BERT_INIT_CHKPNT = '../input/uncased-l12-h768-a12/bert_model.ckpt'
BERT_CONFIG = '../input/uncased-l12-h768-a12/bert_config.json'
#BERT_CONFIG = '/content/drive/MyDrive/685-NLP_Project/bert_config.json'

In [ ]:
BERT_VOCAB= '/content/drive/MyDrive/685-NLP_Project/cased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = '/content/drive/MyDrive/685-NLP_Project/cased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = '/content/drive/MyDrive/685-NLP_Project/cased_L-12_H-768_A-12/bert_config.json'

In [ ]:
#tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

In [ ]:
#bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

print(BERT_CONFIG)
model_fn = model_fn_builder(
  bert_config=bert_config,
  num_labels= len(LABEL_COLUMNS),
  init_checkpoint=BERT_INIT_CHKPNT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=False,
  use_one_hot_embeddings=False)

print(LABEL_COLUMNS)
estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

/content/drive/MyDrive/685-NLP_Project/cased_L-12_H-768_A-12/bert_config.json
['Loaded Language', 'Name calling/Labeling', 'Smears', 'Appeal to fear/prejudice', 'Black-and-white Fallacy/Dictatorship', 'Exaggeration/Minimisation', 'Slogans', "Misrepresentation of Someone's Position (Straw Man)", 'Obfuscation, Intentional vagueness, Confusion', 'Presenting Irrelevant Data (Red Herring)', 'Whataboutism', 'Causal Oversimplification', 'Appeal to authority', 'Flag-waving', 'Doubt', 'Thought-terminating cliché', 'Bandwagon', 'Reductio ad hitlerum', 'Repetition', 'Glittering generalities (Virtue)']
INFO:tensorflow:Using config: {'_model_dir': '/content/drive/MyDrive/685-NLP_Projectbaseline/output', '_tf_random_seed': None, '_save_summary_steps': 500, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hour

In [ ]:
print(f'Beginning Training!')
current_time = datetime.datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Skipping training since max_steps has already saved.
Training took time  0:00:02.304188


In [ ]:
# Checking Validation Set Accuracy, Uncomment if needed

# eval_file = os.path.join('../working', "eval.tf_record")
# #filename = Path(train_file)
# if not os.path.exists(eval_file):
#     open(eval_file, 'w').close()

# eval_examples = create_examples(x_val)
# file_based_convert_examples_to_features(
#     eval_examples, MAX_SEQ_LENGTH, tokenizer, eval_file)

# # This tells the estimator to run through the entire set.
# eval_steps = None

# eval_drop_remainder = False
# eval_input_fn = file_based_input_fn_builder(
#     input_file=eval_file,
#     seq_length=MAX_SEQ_LENGTH,
#     is_training=False,
#     drop_remainder=False)

# result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

# output_eval_file = os.path.join("../working", "eval_results.txt")
# with tf.gfile.GFile(output_eval_file, "w") as writer:
#     tf.logging.info("***** Eval results *****")
#     for key in sorted(result.keys()):
#         tf.logging.info("  %s = %s", key, str(result[key]))
#         writer.write("%s = %s\n" % (key, str(result[key])))

In [ ]:
from sklearn.metrics import f1_score

THRESHOLD_PROBABILITY = 0.05

print('Begin predictions!')
current_time = datetime.datetime.now()
predictions = estimator.predict(test_input_fn)
#evaluations = estimator.evaluate(input_fn=test_input_fn, steps=None)
print("Predicting took time ", datetime.datetime.now() - current_time)


def create_output(predictions):
    finalPreds = []
    for (i, prediction) in enumerate(evaluations):
        print(prediction)
        preds = prediction["probabilities"]
        preds = np.where(preds >= THRESHOLD_PROBABILITY, 1, 0)
        finalPreds.append(preds)

    dff = pd.DataFrame(finalPreds)

output_df = create_output(predictions)
merged_df =  pd.concat([testingData, output_df], axis=1)
submission = merged_df.drop(['text'], axis=1)
#print(submission)

y_true = (submission.iloc[:192, :20]).to_numpy()
print(y_true)
y_pred = (submission.iloc[:192, 20:]).to_numpy()
print(y_pred)

print(str(f1_score(y_true, y_pred, average="samples")))


Begin predictions!
Predicting took time  0:00:00.000079
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:labels_check1:Tensor("IteratorGetNext:3", shape=(32, 20), dtype=int32)
INFO:tensorflow:labels_check2:Tensor("loss/Cast:0", shape=(32, 20), dtype=float32)
INFO:tensorflow:num_labels:20;logits:Tensor("loss/BiasAdd:0", shape=(32, 20), dtype=float32);labels:Tensor("loss/Cast:0", shape=(32, 20), dtype=float32)
INFO:tensorflow:**** Trainable Variables ****
mode: infer true_labels: Tensor("IteratorGetNext:3", shape=(32, 20), dtype=int32) probabilities: Tensor("loss/Sigmoid:0", shape=(32, 20), dtype=float32)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/685-NLP_Projectbaseline/output/model.ckpt-430
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[[0. 1. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ..

ValueError: ignored

In [ ]:
# Convert the Probabilities from Predictions into Labels:
from sklearn.metrics import f1_score

THRESHOLD_PROBABILITY = 0.05

print('Begin predictions!')
current_time = datetime.datetime.now()
#predictions = estimator.predict(test_input_fn)
evaluations = estimator.evaluate(input_fn=test_input_fn, steps=None)
print("Predicting took time ", datetime.datetime.now() - current_time)

# def create_output(predictions):
#     finalPreds = []
#     for (i, prediction) in enumerate(predictions):
#         print(prediction)
#         preds = prediction["probabilities"]
#         preds = np.where(preds >= THRESHOLD_PROBABILITY, 1, 0)
#         finalPreds.append(preds)

#     dff = pd.DataFrame(finalPreds)

#     print(finalPreds)
#     dff.columns = LABEL_COLUMNS
#     return dff

def get_f1_score(evaluations):
    finalPreds = []
    for (i, prediction) in enumerate(evaluations):
        print(prediction)
        preds = prediction["probabilities"]
        preds = np.where(preds >= THRESHOLD_PROBABILITY, 1, 0)
        finalPreds.append(preds)

    dff = pd.DataFrame(finalPreds)

    print(finalPreds)
    dff.columns = LABEL_COLUMNS
    return dff


#y_pred_df = create_output(predictions)
# y_pred = y_pred_df.to_numpy()
# print(y_pred)
f1_score = get_f1_score(evaluations)



output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                eval_metric_ops=eval_metrics,
                predictions={"probabilities": probabilities},
                scaffold=scaffold_fn)

y_true = []
test_input = test_input_fn({"batch_size": 32})
for x,y in test_input:
  print (y)
  print(_decode_record(record, name_to_features))
  #y_true.append(example.labels)

print(np.shape(y_pred), np.shape(y_true))
#f1_score(y_true, y_pred, average="samples")
# merged_df =  pd.concat([x_test, output_df], axis=1)
# submission = merged_df.drop(['comment_text'], axis=1)
# submission.to_csv("sample_submission.csv", index=False)